In [8]:
%cd /Users/johnmartin/Documents/GraduateSchool/Research/ML_Gravity/

/Users/johnmartin/Documents/GraduateSchool/Research/ML_Gravity


In [9]:
from GravNN.Networks.utils import configure_tensorflow
from GravNN.Networks.Model import load_config_and_model
from GravNN.Networks.Data import standardize_output
tf = configure_tensorflow()
import numpy as np
import pandas as pd

In [10]:
df_file = "Data/Dataframes/useless_061121.data"
df = pd.read_pickle(df_file)
idx = 0

model_id = df["id"].values[idx]
tf.keras.backend.clear_session()
config, model = load_config_and_model(model_id, df)

x_transformer = config['x_transformer'][0]
u_transformer = config['u_transformer'][0]

x_scaler = x_transformer.data_range_
u_scaler = u_transformer.data_range_

In [26]:
a = 6378.0+420.0
e = 0.1
i = np.pi/4.0
omega = 0.0
Omega = 0.0
tau = 0.0
OE = [a, e, i, omega, Omega, tau]
mu = 3.986004418*10**14 / 1000**3 # km^3/s^2

x0 = tf.constant(x_scaler, dtype=tf.float32, name='x_scale')
u0 = tf.constant(u_scaler, dtype=tf.float32, name='u_scale')
mu = tf.constant(mu, dtype=tf.float32, name='mu')

OE = tf.Variable(OE, dtype=tf.float32, name='orbit_elements')

f = tf.constant(0.0, dtype=tf.float32)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(OE)
    tape.watch(x0)
    tape.watch(u0)
    tape.watch(mu)
    tape.watch(f) # Needs to be watched because it isn't a 'trainable variable' https://stackoverflow.com/questions/56916313/tensorflow-2-0-doesnt-compute-the-gradient

    a = OE[0]
    e = OE[1]
    i = OE[2]
    omega = OE[3]
    Omega = OE[4]
    tau = OE[5]

    p = a*(1-e**2)

    e_tensor, e_mag = tf.linalg.normalize(e)

    cf = tf.math.cos(f)
    sf = tf.math.sin(f)

    r_eHatTerm = p*cf/(1.0+e_mag*cf)
    r_ePerpHatTerm = p*sf/(1.0+e_mag*cf)

    v_eHatTerm = tf.math.sqrt(mu/p)*-sf
    v_ePerpHatTerm = tf.math.sqrt(mu/p)*(e_mag+cf)

    cw = tf.math.cos(omega)
    sw = tf.math.sin(omega)

    cO = tf.math.cos(Omega)
    sO = tf.math.sin(Omega)

    ci = tf.math.cos(i)
    si = tf.math.sin(i)

    r_nHat = r_eHatTerm*tf.stack([cw,  sw], 0) + r_ePerpHatTerm*tf.stack([-1.0*sw, cw], 0)
    r_xyz = r_nHat[0]*tf.stack([cO, sO, 0.0], 0) + r_nHat[1]*tf.stack([-1.0*ci*sO, ci*cO, si], 0)

    v_nHat = v_eHatTerm*tf.stack([cw,  sw], 0) + v_ePerpHatTerm*tf.stack([-1.0*sw, cw], 0)
    v_xyz = v_nHat[0]*tf.stack([cO, sO, 0.0], 0) + v_nHat[1]*tf.stack([-1.0*ci*sO, ci*cO, si], 0)

    
    output = tf.stack([cw,  sw])
    output = tf.concat([cw, sw], 0)
    
    r, v = r_xyz, v_xyz
    r_scaled = tf.reshape(r/x0, shape=(1,3))
    y_hat = model(r_scaled)#.astype('float32'))
    u_pred, a_pred, laplace_pred, curl_pred = standardize_output(y_hat, config)
    u = u_pred*u0

tape.gradient(r, OE)



<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 8.9999998e-01, -6.7979995e+03,  0.0000000e+00,  8.6524404e+03,
        6.1181997e+03,  0.0000000e+00], dtype=float32)>